# Run pyCIAM

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
from pyCIAM.run import execute_pyciam
from shared import (
    AUTHOR,
    CONTACT,
    DIR_SCRATCH,
    HISTORY,
    PATH_OUTPUTS,
    PATH_PARAMS,
    PATH_REFA,
    PATH_SLIIDERS,
    PATH_SLIIDERS_SEG,
    PATH_SLR_INT,
    PATHS_SURGE_LOOKUP,
    RES_VERS,
    start_dask_cluster,
)

In [5]:
TMPPATH = DIR_SCRATCH / f"pyCIAM_results_quantiles_prechunked_{RES_VERS}.zarr"

# When running on larger/scalable dask cluster, may wish to specify number of workers
# Default is LocalCluster which will use the number of CPUs available on local machine
# N_WORKERS = None
N_WORKERS = 400

SEG_CHUNKSIZE = 3

SEG_ADM_SUBSET = None

DESCRIPTION = "Projected coastal damages from pyCIAM, using quantiles of SLR scenarios."

In [7]:
client, cluster = start_dask_cluster()
cluster.scale(N_WORKERS)
client

Connection method: Cluster object,Cluster type: dask_gateway.GatewayCluster
Dashboard: /services/dask-gateway/clusters/jhub.819a47100b9d4490886627b4ccbc0273/status,


In [11]:
execute_pyciam(
    PATH_PARAMS,
    PATH_SLIIDERS,
    [PATH_SLR_INT],
    ["is"],
    PATH_REFA,
    econ_input_path_seg=PATH_SLIIDERS_SEG,
    output_path=PATH_OUTPUTS if SEG_ADM_SUBSET is None else None,
    seg_var_subset=SEG_ADM_SUBSET,
    surge_input_paths=PATHS_SURGE_LOOKUP,
    tmp_output_path=TMPPATH,
    quantiles=[0.5],
    dask_client_func=lambda: client,
    remove_tmpfile=False,
    extra_attrs={
        "author": AUTHOR,
        "contact": CONTACT,
        "description": DESCRIPTION,
        "history": HISTORY,
    },
)

/srv/conda/envs/notebook/lib/python3.12/site-packages/distributed/client.py:3157: UserWarning: Sending large graph of size 10.33 MiB.
This may cause some slowdown.
Consider scattering data ahead of time and using futures.
  warnings.warn(
